In [1]:
import os

In [2]:
%pwd

'c:\\Users\\kavya\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\kavya\\Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Nisargak18/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Nisargak18"
os.environ["MLFLOW_TRACKING_PASSWORD"]="279be89920dbbb7d5639153c2458795665c0d2d1"


In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Nisargak18/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import shutil




In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)  #FIXED HERE
        self.save_score()


    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    
    def log_into_mlflow(self):

        
        
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })

            # Save model to a temp dir
            model_dir = "temp_model_dir"
            model_path = os.path.join(model_dir, "model.h5")
            os.makedirs(model_dir, exist_ok=True)
            self.model.save(model_path)

            # Log model as artifact (not as registered model)
            mlflow.log_artifact(model_path, artifact_path="VGG16Model")

            # Clean up
            shutil.rmtree(model_dir)


SyntaxError: invalid decimal literal (3152393831.py, line 52)

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-07-09 10:51:00,698: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-09 10:51:00,710: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-09 10:51:00,715: INFO: common: created directory at: artifacts]


Found 139 images belonging to 2 classes.

[2025-07-09 10:51:04,220: WARNING: module_wrapper: From c:\Users\kavya\miniconda3\envs\kidney\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
9/9 [==============================] - 42s 4s/step - loss: 0.2443 - accuracy: 0.8777
[2025-07-09 10:51:44,687: INFO: common: json file saved at: scores.json]


c:\Users\kavya\miniconda3\envs\kidney\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


🏃 View run victorious-skink-588 at: https://dagshub.com/Nisargak18/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0/runs/e552f028f3444a37af28d10a61254e46
🧪 View experiment at: https://dagshub.com/Nisargak18/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0
